In [1]:
import pandas as pd

import io
 
df = pd.read_csv("ais_disabling_events_main_csv.csv")
print(df.head())

                             gap_id       mmsi vessel_class flag  \
0  40072d9edfca0cfd28872cbf9e382e0c  111112726        other  NaN   
1  52dcbd77bbdad9ee2642f574acf45190  150402949        other  CHN   
2  22b6d49ed4bb4d017884b6299ed185f4  204201370        other  PRT   
3  1abe463d7f61b110c44f1d1b4654a213  204206740        other  PRT   
4  72d047a15e33f67c3c5b91c78ff612be  204209840        other  PRT   

   vessel_length_m  vessel_tonnage_gt        gap_start_timestamp  \
0        32.064056         139.583282  2018-03-22 08:59:22+00:00   
1        54.994278         904.204044  2019-01-31 04:09:19+00:00   
2        15.350000          40.940000  2018-05-27 04:47:20+00:00   
3        20.000000          77.263333  2019-06-21 12:17:01+00:00   
4        20.000000          87.435000  2017-07-02 13:13:08+00:00   

   gap_start_lat  gap_start_lon  gap_start_distance_from_shore_m  \
0      11.847433     -21.832185                           384000   
1     -45.895618     -60.642950               

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (17,18) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


### **AIS Disabling in EEZ**

In [2]:
# did ais get disabled in eez (dist from shore) (check length of eez from google 200 nautical miles to km)

gap_start_distance_from_shore_m = df['gap_start_distance_from_shore_m'].tolist()

eez_check = []
for x in gap_start_distance_from_shore_m:
  if x > 370400:
    eez_check.append(0)
  else:
    eez_check.append(1)


In [3]:
import pandas as pd

column_name = "eez_check"

df_eez = pd.DataFrame(eez_check, columns = [column_name])
df_eez.head()

df_eez.to_csv("eez_check.csv")

### **Distance Traversed during AIS disabling event**

In [4]:
# distance traversed during ais disabling event

import numpy as np
import pandas as pd
from math import radians, cos, sin, asin, acos, sqrt, pi
from geopy import distance
from geopy.geocoders import Nominatim
import networkx as nx

def calculate_spherical_distance(lat1, lon1, lat2, lon2, r=6371):
    # Convert degrees to radians
    coordinates = lat1, lon1, lat2, lon2
    # radians(c) is same as c*pi/180
    phi1, lambda1, phi2, lambda2 = [
        radians(c) for c in coordinates
    ]  
    
    # Apply the haversine formula
    a = (np.square(sin((phi2-phi1)/2)) + cos(phi1) * cos(phi2) * 
         np.square(sin((lambda2-lambda1)/2)))
    d = 2*r*asin(np.sqrt(a))
    return d



In [5]:
gap_start_lat = list(df['gap_start_lat'])
gap_start_lon = list(df['gap_start_lon'])
gap_end_lat = list(df['gap_end_lat'])
gap_end_lon = list(df['gap_end_lon'])

spherical_distances = []

for i in range(len(gap_start_lat)):
  spherical_distances.append(calculate_spherical_distance(
      gap_start_lat[i],
      gap_start_lon[i],
      gap_end_lat[i],
      gap_end_lon[i]
  ))


In [6]:
spherical_distances[:5]

[207.94284503711143,
 9.704231955303069,
 10.789069356511098,
 11.6958596191019,
 93.1133345479983]

In [8]:
import pandas as pd

column_name = "spherical_distances"

df_dist = pd.DataFrame(spherical_distances, columns = [column_name])
df_dist.head()

df_dist.to_csv("distance_traversed.csv")

### **CHECK POINT IN FAO JSON**

In [ ]:
!pip install streamlit
!pip install geojson

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 64.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 KB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.1/238.1 KB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.0/184.0 KB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.0/79.0 KB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 73.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.1/51.1 KB 6.3 MB/s eta 0:00:00
  Created wheel for validators: filename=validators-0.20.0-py3-none-any.whl size=19581 sha256=e927957c9dd7ec206ecc5ef7f7263478b65f6375b7b24083ab56dbdff8155f65
  Stored in directory: /root/.cache/pip/wheels/19/09/72/3eb7

In [ ]:
# import streamlit as st
import pandas as pd
import geojson
import folium
# from streamlit_folium import st_folium
from shapely.geometry import Point, MultiPolygon
import re

with open('FAO_AREAS_CWP.json') as f:
    geojson_file = geojson.load(f)

In [ ]:
from shapely.geometry import Point, Polygon

ocean_list = []

def identify_ocean(longitude, latitude):
  # Creating a Shapely point from the longitude and latitude (first long then lat VERY IMP)
  point = Point(longitude, latitude)

  # Iterating through the features in the GeoJSON file
  for feature in geojson_file['features']:
    # Checking if the point is within the polygon defined by the feature
    if Point(point).within(Polygon(feature['geometry']['coordinates'][0][0])):
      print(feature['properties']['OCEAN'])
      return feature['properties']['OCEAN']
  
  # If point is in none of the oceans return false
  return False

# make list of all the lat and long of all the ais disabling events
gap_start_lat = df['gap_start_lat'].tolist()
gap_start_lon = df['gap_start_lon'].tolist()

for i in range(len(gap_start_lon)):
  temp = identify_ocean(gap_start_lon[i], gap_start_lat[i])
  ocean_list.append(temp)


print(ocean_list[:5])

Streaming output truncated to the last 5000 lines.
Pacific
Pacific
Pacific
Pacific
Pacific
Pacific
Pacific
Pacific
Pacific
Pacific
Pacific
Pacific
Pacific
Pacific
Pacific
Pacific
Pacific
Pacific
Pacific
Pacific
Pacific
Pacific
Pacific
Pacific
Pacific
Pacific
Pacific
Pacific
Pacific
Pacific
Pacific
Atlantic
Pacific
Pacific
Pacific
Pacific
Pacific
Pacific
Pacific
Atlantic
Atlantic
Pacific
Atlantic
Indian
Pacific
Pacific
Pacific
Indian
Indian
Indian
Pacific
Indian
Indian
Indian
Indian
Pacific
Indian
Indian
Pacific
Pacific
Pacific
Pacific
Pacific
Pacific
Pacific
Atlantic
Atlantic
Atlantic
Atlantic
Atlantic
Pacific
Indian
Indian
Indian
Indian
Indian
Indian
Indian
Indian
Pacific
Pacific
Pacific
Pacific
Pacific
Pacific
Pacific
Pacific
Pacific
Pacific
Pacific
Pacific
Pacific
Pacific
Atlantic
Atlantic
Pacific
Pacific
Pacific
Pacific
Pacific
Pacific
Indian
Indian
Indian
Indian
Indian
Pacific
Pacific
Pacific
Pacific
Atlantic
Atlantic
Pacific
Pacific
Indian
Pacific
Pacific
Pacific
Pacific
Pacific


In [ ]:
import pandas as pd

column_name = "oceans_list"

df = pd.DataFrame(ocean_list, columns = [column_name])
df.head()

df.to_csv("csv_name.csv")